# Решение #3 - Ансамбль

## Содержание
* [Описание решения](#description)
* [Подготовительные действия](#preprocess)
* [Решение](#solution)
* [Результаты](#result)

## Описание решения <a id='description'></a>

### Решение основано на методе голосования, где каждый алгоритм голосует за один из ответов и каждый алгоритм имеет свой вес
### Порядок действий
1) Предобработка строк
2) Назначение своего веса каждому алгоритму
3) Выбор 2 уникальных не сгруппированных строк
4) Подсчет голосов каждого алгоритма
5) Если сумма голосов больше 0, то считаем, что это рерайт и группируем их
6) Повтор действий 3-5 до тех пор пока каждая строка не будет находится в группе
### Использованные алгоритмы:
* Косинусное сходство с векторзацией подсчетом
* Косинусное сходство с векторизацией tf-idf
* Расстояние Левенштейна
* Расстояние Дамерау-Левенштейна
* Манхэттонское расстояние
* Евклидово расстояние
* Алгоритм выравнивания последовательности в сочетании с расстоянием Хэмминга
* Расстояние Хэмминга 
* Расстояние Хэллингера
* Индекс Жаккарда
* Расстояние Джаро-Винклера
* Дивергенция Дженсена-Шеннона
* Расстояние Карловского
* Алгоритм Майерса
* N-граммы
* N-граммы в сочетании с индексом Тверского
* Коэффициент Сёренсена
* Сравнение множеств слов
* Сравнение наборов местоимений

## Подготовительные действия <a id='preprocess'></a>

### Импортирование библиотек и загрузка датасета

In [1]:
from src import *
import pandas as pd
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

df = pd.read_json('./../data/sample.json')

### Предобработка строк

In [2]:
df['text_preprocess'] = df['text'].apply(load_dataset.preprocess_text)

### Полученный после предобработки датасет

In [3]:
df.head()

,id,text,text_preprocess
0,1,Ты нашёл их или нет?,ты нашел их или нет
1,2,Почему она так со мной поступает?,почему она так со мной поступает
2,3,Никто туда больше не ходит.,никто туда больше не ходит
3,4,У него с собой не было тогда денег.,у него с собой не было тогда денег
4,5,Почему они с нами так поступают?,почему они с нами так поступают


## Решение <a id='solution'></a>

### Вспомогательный класс, который хранит указатель на функцию, пороговое значение и вес функции

In [4]:
class Function:
    def __init__(self,function,threshold:float,weight:int):
        self.function = function
        self.threshold = threshold
        self.weight = weight
    
    def vote(self,left,right):
        if self.threshold == 0:
            if self.function(left,right):
                return self.weight
            else:
                return -self.weight
        else:
            if self.function(left,right,self.threshold):
                return self.weight
            else:
                return -self.weight

### Основной блок решения

In [5]:
# Веса голосов для алгоритмов
LOW = 1
MEDIUM = 2
HIGH = 3
EXTREME = 5

# Пороговое значение (Если строки совпадают на значение больше, чем threshold, то строки считаются рерайтом)
threshold = 0.85

# Функции, участвующие в ансамбле
functions = [Function(cosine_count.is_rewrite_cosine_count,threshold,MEDIUM),
            Function(cosine_tfidf.is_rewrite_cosine_tfidf,threshold,MEDIUM),
            Function(lev_dist.is_rewrite_levenshtein_distance,threshold,LOW),
            Function(dam_lev_dist.is_rewrite_damerau_levenshtein_distance,threshold,LOW),
            Function(distance_L1.is_rewrite_distance_L1,threshold,LOW),
            Function(distance_L2.is_rewrite_distance_L2,threshold,LOW),
            Function(hamming_distance_and_sequence_alignment_composition.is_rewrite_hamming_distance_normalized,threshold,LOW),
            Function(hamming_distance_custom.is_rewrite_hamming_distance_custom,threshold,LOW),
            Function(hellinger.is_rewrite_hellinger_distance,threshold,LOW),
            Function(jaccard.is_rewrite_Jaccard,threshold,MEDIUM),
            Function(jaro_winkler.is_rewrite_jaro_winkler,threshold,MEDIUM),
            Function(JSD.is_rewrite_JSD,threshold,MEDIUM),
            Function(karlovskiy_distance.is_rewrite_karlovskiy_distance,threshold,HIGH),
            Function(myers.is_rewrite_myers,threshold,LOW),
            Function(ngrams.is_rewrite_ngram,threshold,HIGH),
            Function(ngrams.is_rewrite_tverskiy_ngram,threshold,HIGH),
            Function(sorensen.is_rewrite_Sorensen,threshold,MEDIUM),
            Function(word_set.is_rewrite_word_set,threshold,MEDIUM),
            Function(pronouns.check_pronoun_correspondence,0,EXTREME)]

# Убираем лишние предупреждения
pd.options.mode.chained_assignment = None 

df['used'] = [False] * len(df)
groups = []

# Цикл деления строк на группы
for i in tqdm(range(len(df))):
    if df['used'][i]:  # Проверка метки использования строки
        continue
        
    groups.append([])
    groups[-1].append(df['text'][i])
    df['used'][i] = True
    
    for j in range(i+1,len(df)):
        if df['used'][j]: # Проверка метки использования строки
            continue
        
        # Подсчет голосов
        votes = sum([func.vote(df['text_preprocess'][i],df['text_preprocess'][j]) for func in functions])
        
        if(votes > 0):
            groups[-1].append(df['text'][j])
            df['used'][j] = True

    # Если строка не имеет рерайта, то группа не формируется
    if len(groups[-1]) == 1: 
        df['used'][i] = False 
        groups = groups[:-1]


# Добавляем уникальные строки (не имеющие рерайт)
groups = groups + [[str] for str in list(df[df['used']==False]['text'])]

df = df.drop('used',axis=1)

  0%|          | 0/412 [00:00<?, ?it/s]

In [6]:
# Проверка, что строки не были взяты дважды
assert(sum([len(group) for group in groups]) == len(df)) 

## Результаты <a id='result'></a>

### Все найденные группы

In [7]:
print('Количество групп = ',len(groups))

Количество групп =  280


In [8]:
groups

[['Почему она так со мной поступает?', 'Почему он так со мной поступает?'],
 ['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Почему она так с ней поступает?', 'Почему она так с ним поступает?'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влер

### Строки, которые имеют более 1 рерайта

In [9]:
groups3 = [group for group in groups if len(group) > 2]

In [10]:
print('Количество групп = ',len(groups3))

Количество групп =  1


In [11]:
groups3

[['Том не хочет никого видеть.',
  'Том не хочет сегодня никого видеть.',
  'Том сегодня не хочет никого видеть.']]

### Строки, которые имеют ровно 1 рерайт

In [12]:
groups2 = [group for group in groups if len(group) == 2]

In [13]:
print('Количество групп = ',len(groups2))

Количество групп =  130


In [14]:
groups2

[['Почему она так со мной поступает?', 'Почему он так со мной поступает?'],
 ['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Почему она так с ней поступает?', 'Почему она так с ним поступает?'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влер

### Строки, которые не имеют рерайта

In [15]:
groups1 = [group for group in groups if len(group) == 1]

In [16]:
print('Количество групп = ',len(groups1))

Количество групп =  149


In [17]:
groups1

[['Ты нашёл их или нет?'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том этим сейчас занимается.'],
 ['Том был просто не готов.'],
 ['Я ничего не хочу делать.'],
 ['Тому это тоже не нравится.'],
 ['Сколько сейчас времени в Париже?'],
 ['Тому было больше не к кому обратиться.'],
 ['Он сказал, что вчера был дома.'],
 ['Они никогда меня не слушают.'],
 ['Я поймал сегодня три рыбы.'],
 ['Я хочу что-нибудь сделать для Тома.'],
 ['Я сделал всё правильно?'],
 ['Я тоже завтра пойду в университет.'],
 ['Мне никогда не нужно было столько денег.'],
 ['Том не может жить без Мэри.'],
 ['Почему они так с ним поступают?'],
 ['Что могло с ними случиться?'],
 ['Мы будем делать всё по-своему.'],
 ['Где был Том весь день?'],
 ['Я здесь часто ем.'],
 ['Я всё ей рассказываю.'],
 ['Том больше мне не доверяет.'],
 ['Какая гора с

### Сохранение результата

In [18]:
save_groups.save_groups(groups,"../output/solution3-result.json")